In [1]:
from itertools import chain
from src_draft.utils import LOW_IMP_FEATURES, ZERO_IMP_FEATURES
import shared.ml_config_core as ml_config_core
import pandas as pd
from shared.ml_config_core import ModelConfigsCollection
from shared.ml_config_runner import run_tuning_for_configs_collection
from shared.definitions import TuningResult

from Draft import feature_builder_v2

import importlib

VERBOSE = True
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE)
display("Distribution of Samples")
value_counts = features_matrix["TARGET"].value_counts()
df_value_counts = pd.DataFrame(
    {"Total Rows": value_counts, "Proportion": value_counts / len(features_matrix)}
)
df_value_counts.round(2)
transformers = [
    # No feature engineering tuning for for rejected/accepted df
    # ml_config_core.AcceptedRejectedDropAllButEmployment,  # Only used for testing
]

baseline_only_credit_ratings_config = {
    # Only uses  ExtScore1/2/3
    "Baseline_Only_CreditRatings": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS
        })
}

baseline_only_base_feats = {
    # Only includes base table (i.e. no aggregations from client history)
    "LGBM_AUC_Base_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE,
        }),
}

weighted_log_loss_config = {
    # Tuned on score combined from logloss and F1 score
    "LGBM_Weighted_LogLoss": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneWeightedLogLossF1(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_config: ModelConfigsCollection = {
    "LGBM_AUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

dart_tune_auc_config: ModelConfigsCollection = {
    "LGBM_Dart_AUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMDartTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_all_features_config: ModelConfigsCollection = {
    "LGBM_AUC_All_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            # "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

configs: ModelConfigsCollection = {
    # **baseline_only_credit_ratings_config,
    # **baseline_only_base_feats,
    # **weighted_log_loss_config,
    **dart_tune_auc_config,
    **tune_auc_config,
    # **tune_auc_all_features_config
}

C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Full DS size: 307511


'Distribution of Samples'

C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [2]:
configs.keys()

dict_keys(['LGBM_Dart_AUC', 'LGBM_AUC'])

In [3]:
tuning_results = run_tuning_for_configs_collection(configs, feature_builder_v2.load_datasets_and_prepare_features)

C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history


[I 2024-04-20 16:49:35,795] A new study created in memory with name: no-name-6b22f793-987e-47dc-a5ae-322642fa7e52


drop drop_cols_post_proc: 227
after drop_cols_post_proc: 121
Full DS size: 307511
Tunning: - transformers: transformers: 0
 total options: 0
[]
search_grid:
{}

 - model: LGBM_Dart_AUC n_iters=10 with:
 {'model__n_estimators': Range(50, 1000, 50, int), 'model__learning_rate': Range(0.01, 0.3, 0.01, float), 'model__max_depth': Range(3, 11, 1, int), 'model__num_leaves': Range(8, 256, 8, int), 'model__min_gain_to_split': Range(0.0, 15.0, 0.5, float), 'model__min_data_in_leaf': Range(0, 3000, 100, int), 'model__lambda_l1': Range(0, 110, 5, int), 'model__lambda_l2': Range(0, 110, 5, int), 'model__bagging_fraction': Range(0.2, 1.0, 0.1, float), 'model__feature_fraction': Range(0.2, 1.0, 0.1, float), 'model__max_bin': Range(50, 500, 25, int)}


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\lightgbm\callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


...

Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Tune: val_score:0.7738, std_test_score:0.00350 train_set_score:0.7977
folds val/train: [0.7741, 0.7708, 0.7713, 0.7772, 0.7805, 0.7718, 0.7706] / [0.798, 0.799, 0.7998, 0.7959, 0.797, 0.7951, 0.7993], mean fold time: 3.03


[I 2024-04-20 20:43:26,695] Trial 78 finished with value: 0.773772370853644 and parameters: {'model__n_estimators': 650, 'model__learning_rate': 0.11, 'model__max_depth': 3, 'model__num_leaves': 224, 'model__min_gain_to_split': 3.5, 'model__min_data_in_leaf': 2300, 'model__lambda_l1': 65, 'model__lambda_l2': 100, 'model__bagging_fraction': 0.6000000000000001, 'model__feature_fraction': 0.9000000000000001, 'model__max_bin': 475}. Best is trial 14 with value: 0.7754762116288143.


Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Tune: val_score:0.7745, std_test_score:0.00359 train_set_score:0.8071
folds val/train: [0.7745, 0.7716, 0.7712, 0.7791, 0.7807, 0.7723, 0.7718] / [0.8045, 0.8098, 0.8043, 0.8102, 0.8046, 0.8053, 0.8108], mean fold time: 4.41


[I 2024-04-20 20:43:58,340] Trial 79 finished with value: 0.7744567878266669 and parameters: {'model__n_estimators': 650, 'model__learning_rate': 0.060000000000000005, 'model__max_depth': 4, 'model__num_leaves': 240, 'model__min_gain_to_split': 0.5, 'model__min_data_in_leaf': 2400, 'model__lambda_l1': 80, 'model__lambda_l2': 70, 'model__bagging_fraction': 0.30000000000000004, 'model__feature_fraction': 0.6000000000000001, 'model__max_bin': 450}. Best is trial 14 with value: 0.7754762116288143.



 -- -- --
 TUNING RESULT: 
 LGBM_AUC: auc = 0.7754762116288143 params: {'model__n_estimators': 1000, 'model__learning_rate': 0.060000000000000005, 'model__max_depth': 5, 'model__num_leaves': 248, 'model__min_gain_to_split': 2.5, 'model__min_data_in_leaf': 2400, 'model__lambda_l1': 35, 'model__lambda_l2': 40, 'model__bagging_fraction': 0.2, 'model__feature_fraction': 0.8, 'model__max_bin': 500}
 -- --

LGBM_AUC Fit, total time:2109.6



In [13]:
for t, v in tuning_results.items():
    display(t)
    display(v.hyper_param_all_cv_results)



'LGBM_Dart_AUC'

,Trial Number,Mean Test Score (AUC),mean_test_score,mean_train_score,std_test_score,macro_f1,micro_f1,f1_target1,precision_target1,recall_target1,log_loss,pr_auc,mean_fold_time,Parameters
74,74,0.776747,0.776747,0.830984,0.003274,0.563643,0.734462,0.290626,0.185269,0.673797,0.535689,0.265447,23.572300,"{'model__boosting_type': 'dart', 'model__objec..."
75,75,0.776593,0.776593,0.831826,0.003579,0.563294,0.734393,0.289944,0.184870,0.671742,0.534938,0.266426,23.550009,"{'model__boosting_type': 'dart', 'model__objec..."
66,66,0.776588,0.776588,0.830350,0.003268,0.562434,0.733112,0.289152,0.184177,0.672387,0.536449,0.265445,25.292503,"{'model__boosting_type': 'dart', 'model__objec..."
53,53,0.776571,0.776571,0.835537,0.003876,0.563642,0.735268,0.289980,0.185059,0.669648,0.533383,0.265848,29.795808,"{'model__boosting_type': 'dart', 'model__objec..."
61,61,0.776485,0.776485,0.823652,0.003715,0.561293,0.730559,0.288790,0.183496,0.677623,0.540832,0.265591,25.213860,"{'model__boosting_type': 'dart', 'model__objec..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,0.763828,0.763828,0.780082,0.003558,0.544386,0.705672,0.273306,0.170671,0.685599,0.575386,0.247912,7.348157,"{'model__boosting_type': 'dart', 'model__objec..."
1,1,0.763399,0.763399,0.799970,0.003901,0.548395,0.712277,0.276348,0.173377,0.680524,0.565564,0.246568,1.834589,"{'model__boosting_type': 'dart', 'model__objec..."
9,9,0.759342,0.759342,0.780662,0.003533,0.542149,0.703402,0.270432,0.168719,0.680926,0.594038,0.241080,17.656200,"{'model__boosting_type': 'dart', 'model__objec..."
59,59,0.756181,0.756181,0.778146,0.003688,0.540110,0.700619,0.268418,0.167191,0.680322,0.589754,0.236230,38.954780,"{'model__boosting_type': 'dart', 'model__objec..."


'LGBM_AUC'

,Trial Number,Mean Test Score (AUC),mean_test_score,mean_train_score,std_test_score,macro_f1,micro_f1,f1_target1,precision_target1,recall_target1,log_loss,pr_auc,mean_fold_time,Parameters
14,14,0.775476,0.775476,0.823154,0.003640,0.559619,0.728325,0.287048,0.182104,0.677462,0.540545,0.263373,4.263146,"{'model__objective': 'binary', 'model__class_w..."
43,43,0.775427,0.775427,0.825715,0.003278,0.560401,0.729642,0.287641,0.182678,0.676133,0.539546,0.264757,3.921048,"{'model__objective': 'binary', 'model__class_w..."
21,21,0.775202,0.775202,0.817431,0.003338,0.558119,0.725493,0.286164,0.181100,0.681571,0.544117,0.262951,4.297749,"{'model__objective': 'binary', 'model__class_w..."
68,68,0.775146,0.775146,0.820013,0.003356,0.558903,0.726878,0.286702,0.181649,0.679919,0.542659,0.262858,4.943454,"{'model__objective': 'binary', 'model__class_w..."
67,67,0.775118,0.775118,0.814689,0.003698,0.557178,0.724000,0.285382,0.180399,0.682659,0.545808,0.263104,4.407532,"{'model__objective': 'binary', 'model__class_w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,29,0.759169,0.759169,0.779126,0.003418,0.541233,0.701191,0.270339,0.168358,0.685680,0.579674,0.240910,4.282165,"{'model__objective': 'binary', 'model__class_w..."
52,52,0.754422,0.754422,0.769147,0.003537,0.536445,0.695026,0.265317,0.164686,0.682135,0.605123,0.234764,1.449826,"{'model__objective': 'binary', 'model__class_w..."
39,39,0.754239,0.754239,0.769312,0.021872,0.537541,0.696375,0.266517,0.165543,0.683303,0.603860,0.220045,4.363283,"{'model__objective': 'binary', 'model__class_w..."
57,57,0.746955,0.746955,0.760794,0.003926,0.531099,0.687982,0.259876,0.160713,0.678550,0.663753,0.216402,1.490487,"{'model__objective': 'binary', 'model__class_w..."


In [5]:
tuning_results.keys()

dict_keys(['LGBM_Dart_AUC', 'LGBM_AUC'])

In [6]:
# Results:

# Only main dataset:
# - top res      : 0.753
# - full + gbdt  : 0.765
# - full + dart  : 0.766


In [7]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history
Full DS size: 307511


In [8]:
features_matrix_full

,TARGET,NameContractType,CodeGender,FlagOwnCar,FlagOwnRealty,CntChildren,AmtIncomeTotal,AmtCredit,AmtAnnuity,AmtGoodsPrice,...,PrevCodeRejectReasonHcCount,PrevCodeRejectReasonLimitCount,PrevCodeRejectReasonScoCount,PrevCodeRejectReasonScofrCount,PrevCodeRejectReasonSystemCount,PrevCodeRejectReasonVerifCount,PrevCodeRejectReasonXapCount,PrevCodeRejectReasonXnaCount,PrevLastLoanNflagInsuredOnApproval,PrevAvgNflagInsuredOnApproval
0,1,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,0,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.666667
2,0,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,0,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,297000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,NaN,0.000000
4,0,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,Cash loans,M,False,False,0,157500.0,254700.0,27558.0,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
307507,0,Cash loans,F,False,True,0,72000.0,269550.0,12001.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
307508,0,Cash loans,F,False,True,0,153000.0,677664.0,29979.0,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.500000
307509,1,Cash loans,F,False,True,0,171000.0,370107.0,20205.0,319500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.500000


In [9]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)
features_matrix_base = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.BASE)


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history
Full DS size: 307511
Full DS size: 307511


In [10]:
features_matrix_full

,TARGET,NameContractType,CodeGender,FlagOwnCar,FlagOwnRealty,CntChildren,AmtIncomeTotal,AmtCredit,AmtAnnuity,AmtGoodsPrice,...,PrevCodeRejectReasonHcCount,PrevCodeRejectReasonLimitCount,PrevCodeRejectReasonScoCount,PrevCodeRejectReasonScofrCount,PrevCodeRejectReasonSystemCount,PrevCodeRejectReasonVerifCount,PrevCodeRejectReasonXapCount,PrevCodeRejectReasonXnaCount,PrevLastLoanNflagInsuredOnApproval,PrevAvgNflagInsuredOnApproval
0,1,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,0,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.666667
2,0,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,0,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,297000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,NaN,0.000000
4,0,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,Cash loans,M,False,False,0,157500.0,254700.0,27558.0,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
307507,0,Cash loans,F,False,True,0,72000.0,269550.0,12001.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
307508,0,Cash loans,F,False,True,0,153000.0,677664.0,29979.0,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.500000
307509,1,Cash loans,F,False,True,0,171000.0,370107.0,20205.0,319500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.500000


In [11]:
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS)
from shared import stats_utils

stats_utils.nan_summary(features_matrix)

Full DS size: 307511


,Total NaN Values,Proportion NaN (%)
ExtSource1,173378,56.0
ExtSource2,660,0.0
ExtSource3,60965,20.0
